## 날짜별 워드 클라우드로 가장 빈도수 높은 단어 도출 (title)
- 이예빈
- 참고: 불용어 사전 (https://www.ranks.nl/stopwords/korean)

In [6]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import numpy as np
import pandas as pd
import os
import re
from konlpy.tag import Okt
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import tqdm

In [7]:
try:
    if not(os.path.isdir('./tokenized_news')):
        os.makedirs(os.path.join('./tokenized_news'))
                    
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!")
        raise

In [8]:
# CSV_List로 만드는 함수
def csv2list(filename):
    lists = []
    file = open(filename,'r',encoding='utf-8')
    while True:
        line = file.readline().rstrip('\n')
        if line:
            lists.append(line)
        else:
            break
    return lists

In [9]:
stop_words = csv2list('../stop_words/stop_words.csv')

In [10]:
### 뉴스 제목에서 기호 제거 & 명사 추출 

def textile_preprocessing(text, okt):
    textile_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', text)
    word_textile = okt.nouns(textile_text) 
    word_textile = [word for word in word_textile if not word in stop_words] # 불용어 제거
    
    return word_textile

In [13]:
### 전처리된 데이터프레임 by_day를 반환합니다. 
## file: csv파일이 들어있는 경로를 지정해주세요. (+ 파일까지 포함되도록!)

def all_preprocessing(file):
    df_file = pd.read_csv(file, encoding='utf-8')
    
#     textile = df_file['text']
#     date_ = df_file['date']
#     df = pd.DataFrame({'text':textile, 'date':date_})
    
    okt = Okt()
    clean_text = []
    for text in df_file['text']:
        if type(text)==str:
            clean_text.append(textile_preprocessing(text, okt))
        else:
            clean_text.append([])
    df['text'] = clean_text
    
    # date 컬럼 -> datetime으로 설정 
    df['date'] = pd.to_datetime(df['date'])
    by_day = df.groupby(df['date']).sum()

    # 딕셔너리 풀기
    news_textile = []
    for i in tqdm(range(len(by_day['text']))):
        news_textile.append(', '.join(by_day['text'][i]))

    by_day['text'] = tqdm(news_textile)
    
    by_day.to_csv('./tokenized_news/tokenized_title_okt_nouns', encoding='utf-8')
    
    return by_day

In [14]:
tmp_df = all_preprocessing('./1-8월 뉴스.csv')

KeyboardInterrupt: 

In [ ]:
tmp_df.to_csv('./tokenized_news/tokenized_title_okt_nouns', encoding='utf-8')

In [29]:
def daily_news_title_wordcloud(file, date_to_search):
    tmp_df = all_preprocessing(file)
    
    cloud = WordCloud(
      font_path='../../fonts/NanumSquareRoundR.ttf'
    , width=800
    , height=600
    , max_words=100
    ).generate(tmp_df['text'][date_to_search])

    plt.figure(figsize=(20, 15))
    plt.imshow(cloud)
    plt.axis('off')
    
    

In [30]:
daily_news_title_wordcloud('./1-8월 뉴스.csv','2020-01-23')

KeyboardInterrupt: 

In [2]:
aa = pd.read_csv('./1-8월 뉴스.csv')
aa.head()

,title,text,url,media,date
0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
1,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\n노동의 미래를 모두 알 수는 없다. 다만 현실에서 일어나...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
2,이곳은…기계의 정확도 높이는 ‘인간부품’ 육성공장 아닐까 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\nㆍAI 학습데이터 가공 기업 ‘크라우드웍스’ 교육장 참관...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
3,"지혜씨, 지훈씨를 부탁해! [인포그래픽]",새로 태어나는 아이들은 줄어들고 어르신들은 계속 나이를 먹어 갑니다. 2020년 새...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
4,"[20대 리포트] “20대, 함부로 판단말라… 내 길 내가 간다”",새천년의 시작과 함께 세상에 나온 2000년대생들이 어느덧 사회 변화를 이끌 20대...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-01-01
